In [ ]:
import http.client
import json

import pandas as pd
from pqdm.threads import pqdm
patient_ids = pd.read_csv("/Users/andrewprokhorenkov/Downloads/remove_patients.csv")
# def query_patient_id(patient_id):
def query_patient_id(v):
    conn = http.client.HTTPSConnection("staging.midrc.org")

    payload = f"{{\n\t\"Level\": \"Patient\",\n\t\"Query\": {{\n\t\t\"PatientID\": \"{v['patient_ids']}\"\n\t}}\n}}"

    print(v.name)

    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer "
        }

    conn.request("POST", "/dicom-server/tools/find", payload, headers)

    res = conn.getresponse()
    data = res.read()

    # print(data.decode("utf-8"))

    return data.decode("utf-8")
patient_data_dicom = patient_ids.apply(lambda v: query_patient_id(v), axis=1)
data = patient_data_dicom.apply(lambda v: v.strip())
data = data[data != '[]']
data.to_csv("/Users/andrewprokhorenkov/Downloads/to_remove_dicom.lst", index=False, header=False)
data = data.apply(lambda v: json.loads(v)[0]).to_list()
# data
def delete_dicom_patient_ids(patient_ids):
    conn = http.client.HTTPSConnection("staging.midrc.org")

    payload = {"Resources": patient_ids}
    payload = json.dumps(payload)

    headers = {
        'cookie': "",
        'Content-Type': "application/json",
        'Authorization': "Bearer "
        }

    conn.request("POST", "/dicom-server/tools/bulk-delete", payload, headers)

    res = conn.getresponse()
    data = res.read()

    print(data.decode("utf-8"))
data.index("81411fbb-fb969add-3ef3fe32-92131d78-b0e8d1a0")
for v in data[150:]:
    print(v)
    delete_dicom_patient_ids([v])

In [ ]:
from functools import partial
import http.client
import json
import socket

from pqdm.threads import pqdm
r = pqdm(range(100000), lambda v: v ** 2, n_jobs=10)
def get_access_token(creds_filepath):
    payload = open(creds_filepath)
    headers = {
        "Content-Type": "application/json"
    }
    conn = http.client.HTTPSConnection("staging.midrc.org")
    conn.request("POST", "/user/credentials/api/access_token", payload, headers)

    res = conn.getresponse()
    data = res.read()

    access_token = json.loads(data.decode("utf-8"))

    return access_token
access_token = get_access_token("/Users/andrewprokhorenkov/.creds/staging.midrc.org.json")
study_ids = json.load(open("./download.json"))
def query_study_id(study_id, access_token):
    conn = http.client.HTTPSConnection("staging.midrc.org")

    headers = {
        'Content-Type': "application/json",
        'Authorization': f"Bearer {access_token}"
        }

    conn.request("GET", f"/dicom-server/studies/{study_id}", headers=headers)

    res = conn.getresponse()
    data = res.read()

    r = json.loads(data.decode("utf-8"))

    return {"study_id": study_id, "response": r}

query_study_id_by_id = partial(query_study_id, access_token=access_token["access_token"])
result = pqdm(study_ids, query_study_id_by_id, n_jobs=10)
study_ids_new = study_ids.copy()
for r in result:
    if type(r) != socket.gaierror:
        study_ids_new.remove(r["study_id"])
    # print(r["study_id"])
study_ids_new

json.dumps(query_study_id_by_id("b553dd1a-686e206e-488477a2-36cb9721-de073138"))
with open("all_studies_in_dicom.json", "w") as f:
    json.dump(result, f, indent=4, default=lambda v: {"!!!": str(v)})